In [2]:
# GOOGLE DRIVE KA PURA TREE BANANE WALA FINAL CODE (Roman Urdu Comments)

from googleapiclient.discovery import build
from google.colab import auth, drive, files
from google.auth import default
from IPython.display import display, HTML
import time

# Step 1: Drive mount karo aur permission lo
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

# File ka naam aur location jahan HTML save hoga
output_file = "/content/drive/MyDrive/Google_Drive_Tree_RomanUrdu.html"
print("Google Drive scan kar raha hun...")

def get_all():
    """Saari files aur folders ek saath laao"""
    items = []
    token = None
    while True:
        result = service.files().list(
            q="trashed = false",
            spaces='drive',
            fields='nextPageToken, files(id,name,mimeType,parents,size)',
            pageSize=1000,
            pageToken=token
        ).execute()

        items.extend(result.get('files', []))
        print(f"Ab tak {len(items)} items mile...", end='\r')
        token = result.get('nextPageToken')
        if not token:
            break
    print(f"\nTotal items mile: {len(items)}")
    return items

# Saari files fetch karo
all_items = get_all()

# ID se jaldi dhundne ke liye dictionary banao
by_id = {item['id']: item for item in all_items}

# Har folder ke andar kon-kon se bachche (children) hain, yeh dikhao
children = {}
for item in all_items:
    for parent_id in item.get('parents', []):
        children.setdefault(parent_id, []).append(item)

# Jo files/folders kisi ke andar nahi hain, woh root hain (My Drive ya Shared Drives)
roots = [item for item in all_items if not item.get('parents') or item['parents'][0] not in by_id]

def sort_kids(kids):
    """Pehle folders, phir files — dono alphabetically"""
    folders = [x for x in kids if x['mimeType'] == 'application/vnd.google-apps.folder']
    files_only = [x for x in kids if x['mimeType'] != 'application/vnd.google-apps.folder']
    folders.sort(key=lambda x: x['name'].lower())
    files_only.sort(key=lambda x: x['name'].lower())
    return folders + files_only

# Har folder ko ek unique ID do taaki collapse/open kar sakein
folder_counter = 0
def new_id():
    global folder_counter
    folder_counter += 1
    return f"f{folder_counter}"

def build(item, prefix="", is_last=True):
    """Tree ki har line yahan banegi"""
    name = item['name']
    file_id = item['id']
    is_folder = item['mimeType'] == 'application/vnd.google-apps.folder'

    # Tree mein └── ya ├── lagao
    connector = "└── " if is_last else "├── "
    indent = prefix + ("    " if is_last else "│   ")

    # File size dikhao (sirf files ke liye)
    size_text = ""
    if not is_folder and 'size' in item:
        size = int(item['size'])
        if size >= 1073741824:
            size_text = f" ({size/1073741824:.2f} GB)"
        elif size >= 1048576:
            size_text = f" ({size/1048576:.1f} MB)"
        elif size > 0:
            size_text = f" ({size//1024} KB)"

    if is_folder:
        folder_html_id = new_id()
        line = f'{prefix}{connector}<span class="fold" onclick="t(\'{folder_html_id}\')">Folder <b>{name}</b></span><br>'
        line += f'<div id="{folder_html_id}" class="sub">'
    else:
        view_link = f"https://drive.google.com/file/d/{file_id}/view"
        download_link = f"https://drive.google.com/uc?id={file_id}&export=download"
        line = f'{prefix}{connector}File <a href="{view_link}" target="_blank" class="fn">{name}</a> ' \
               f'<span class="sz">{size_text}</span> ' \
               f'<a href="{download_link}" download class="dl">[Download]</a><br>'

    # Bachchon ko bhi add karo
    kids = sort_kids(children.get(file_id, []))
    for i, kid in enumerate(kids):
        line += build(kid, indent, i == len(kids)-1)

    if is_folder:
        line += '</div>'
    return line

# Pura tree banao
tree_html = ""
for root in sort_kids(roots):
    if root['id'] == 'root' or root.get('name') == 'My Drive':
        tree_html += '<div class="rootname">My Drive</div>'
    else:
        tree_html += f'<div class="shared">Shared Drive: <b>{root["name"]}</b></div>'
    tree_html += build(root, "", True)

total_items = len(all_items)

# Final HTML (JavaScript mein ${} nahi use kiya taaki Colab mein error na aaye)
html = f'''<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Google Drive Tree - Roman Urdu Version</title>
<style>
  body{{font-family:Segoe UI,Arial;background:#f0f2f5;padding:20px;margin:0}}
  h1{{color:#1967d2;text-align:center}}
  input{{width:100%;padding:16px;font-size:20px;border:3px solid #1967d2;border-radius:12px;box-sizing:border-box;margin:15px 0}}
  .info{{text-align:center;margin:15px;font-size:18px;color:#555}}
  .tree{{background:#fff;padding:30px;border-radius:16px;box-shadow:0 10px 30px rgba(0,0,0,0.15);font-family:Consolas,monospace;font-size:15px;line-height:1.8}}
  .fold{{cursor:pointer;color:#1967d2;font-weight:bold}}
  .fold:hover{{text-decoration:underline}}
  .fn{{color:#1a0dab;text-decoration:none}}
  .fn:hover{{text-decoration:underline}}
  .sz{{color:#777;font-size:0.9em}}
  .dl{{margin-left:15px;color:#c5221f;font-weight:bold}}
  .sub{{padding-left:30px;display:none}}
  .rootname,.shared{{font-size:20px;font-weight:bold;color:#1967d2;margin:20px 0 10px}}
  .hl{{background:#ffeb3b !important;padding:2px 6px;border-radius:4px}}
</style>
</head>
<body>
<h1>Google Drive - Pura Tree (Collapsible)</h1>
<input type="text" placeholder="File ka naam search karo..." autofocus>
<div class="info">Total items: {total_items}</div>
<div class="tree">{tree_html}</div>

<script>
// Folder open/close karne ka function
function t(id){{
  var x = document.getElementById(id);
  if (x.style.display === "none" || x.style.display === "none") {{
    x.style.display = "block";
  }} else {{
    x.style.display = "none";
  }}
}}

var input = document.querySelector('input');
var info = document.querySelector('.info');
var found = 0;

input.oninput = function() {{
  var term = input.value.trim().toLowerCase();
  var names = document.querySelectorAll('.fn');
  found = 0;

  names.forEach(function(el) {{
    var text = el.textContent.toLowerCase();
    if (term && text.includes(term)) {{
      el.classList.add('hl');
      found++;
      // Parent folders bhi khol do taaki result dikhe
      var p = el.parentElement;
      while (p) {{
        if (p.classList && p.classList.contains('sub')) p.style.display = 'block';
        p = p.parentElement;
      }}
    }} else {{
      el.classList.remove('hl');
    }}
  }});

  if (term) {{
    info.textContent = "Found " + found + " result" + (found == 1 ? "" : "s") + ' — "' + input.value + '"';
    if (found > 0) document.querySelector('.hl').scrollIntoView({{behavior:'smooth', block:'center'}});
  }} else {{
    info.textContent = "Total items: {total_items}";
  }}
}};
</script>
</body>
</html>'''

# File save karo aur download karwao
with open(output_file, "w", encoding="utf-8") as f:
    f.write(html)

print("\nHo gaya bhai! Sab kuch perfect!")
print("File save hui →", output_file)
files.download(output_file)
display(HTML(html))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Google Drive scan kar raha hun...
Ab tak 632 items mile...
Total items mile: 632

Ho gaya bhai! Sab kuch perfect!
File save hui → /content/drive/MyDrive/Google_Drive_Tree_RomanUrdu.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
# GOOGLE DRIVE PDF ONLY TREE GENERATOR
# Sirf PDFs aur unke relevant folders scan karega

from googleapiclient.discovery import build
from google.colab import auth, drive, files
from google.auth import default
from IPython.display import display, HTML
import time

# Step 1: Drive mount aur authentication
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

# File ka naam change kar diya
output_file = "/content/drive/MyDrive/Google_Drive_PDFs_Tree.html"
print("Google Drive scan kar raha hun (Sirf PDFs ke liye)...")

def get_all_pdfs_and_folders():
    """Sirf Folders aur PDFs fetch karega"""
    items = []
    token = None
    # Query change ki hai: Sirf Folders YA PDFs layega
    query = "trashed = false and (mimeType = 'application/vnd.google-apps.folder' or mimeType = 'application/pdf')"

    while True:
        result = service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id,name,mimeType,parents,size)',
            pageSize=1000,
            pageToken=token
        ).execute()

        items.extend(result.get('files', []))
        print(f"Ab tak {len(items)} items scan kiye...", end='\r')
        token = result.get('nextPageToken')
        if not token:
            break
    print(f"\nTotal items (Folders + PDFs) mile: {len(items)}")
    return items

# Data fetch karo
raw_items = get_all_pdfs_and_folders()

# --- STEP 2: Cleaning Logic (Khali folders hatana) ---
print("Faltu folders filter kar raha hun...")

# Dictionary banao ID se item dhundne ke liye
item_map = {item['id']: item for item in raw_items}
relevant_ids = set()

# Logic: Sirf wo folders rakho jo kisi PDF tak le jate hain
# Pehle saare PDFs dhundo
pdfs = [item for item in raw_items if item['mimeType'] == 'application/pdf']

# Har PDF ke parents ko trace karke 'relevant_ids' mein daalo
for pdf in pdfs:
    relevant_ids.add(pdf['id']) # PDF khud relevant hai

    # Ab iske parents ko upar tak trace karo
    current_parents = pdf.get('parents', [])
    while current_parents:
        parent_id = current_parents[0] # Usually ek file ka ek hi parent hota hai drive mein
        if parent_id in item_map:
            if parent_id in relevant_ids:
                break # Agar pehle se added hai to loop roko (optimization)
            relevant_ids.add(parent_id)
            # Ab is parent ka bhi parent dhundo
            parent_item = item_map[parent_id]
            current_parents = parent_item.get('parents', [])
        else:
            break

# Final filtered list (Sirf kaam ki cheezain)
all_items = [item for item in raw_items if item['id'] in relevant_ids]
print(f"Filtering ke baad items bache: {len(all_items)}")

# --- STEP 3: Tree Building (Wohi purana logic) ---

by_id = {item['id']: item for item in all_items}
children = {}
for item in all_items:
    for parent_id in item.get('parents', []):
        children.setdefault(parent_id, []).append(item)

# Roots dhundo
roots = [item for item in all_items if not item.get('parents') or item['parents'][0] not in by_id]

folder_counter = 0
def new_id():
    global folder_counter
    folder_counter += 1
    return f"f{folder_counter}"

def sort_kids(kids):
    folders = [x for x in kids if x['mimeType'] == 'application/vnd.google-apps.folder']
    files_only = [x for x in kids if x['mimeType'] != 'application/vnd.google-apps.folder']
    folders.sort(key=lambda x: x['name'].lower())
    files_only.sort(key=lambda x: x['name'].lower())
    return folders + files_only

def build(item, prefix="", is_last=True):
    name = item['name']
    file_id = item['id']
    is_folder = item['mimeType'] == 'application/vnd.google-apps.folder'

    connector = "└── " if is_last else "├── "
    indent = prefix + ("    " if is_last else "│   ")

    size_text = ""
    if not is_folder and 'size' in item:
        size = int(item['size'])
        if size >= 1048576:
            size_text = f" ({size/1048576:.1f} MB)"
        elif size > 0:
            size_text = f" ({size//1024} KB)"

    if is_folder:
        folder_html_id = new_id()
        line = f'{prefix}{connector}<span class="fold" onclick="t(\'{folder_html_id}\')">📂 <b>{name}</b></span><br>'
        line += f'<div id="{folder_html_id}" class="sub">'
    else:
        # PDF Icon aur Red color add kiya hai distinct look ke liye
        view_link = f"https://drive.google.com/file/d/{file_id}/view"
        download_link = f"https://drive.google.com/uc?id={file_id}&export=download"
        line = f'{prefix}{connector}🔴 <a href="{view_link}" target="_blank" class="fn">{name}</a> ' \
               f'<span class="sz">{size_text}</span> ' \
               f'<a href="{download_link}" download class="dl">[Download]</a><br>'

    kids = sort_kids(children.get(file_id, []))
    for i, kid in enumerate(kids):
        line += build(kid, indent, i == len(kids)-1)

    if is_folder:
        line += '</div>'
    return line

tree_html = ""
for root in sort_kids(roots):
    if root['id'] == 'root' or root.get('name') == 'My Drive':
        tree_html += '<div class="rootname">My Drive (PDFs Only)</div>'
    else:
        tree_html += f'<div class="shared">Shared Drive: <b>{root["name"]}</b></div>'
    tree_html += build(root, "", True)

total_items = len(all_items)

# HTML Structure
html = f'''<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Google Drive - PDF Collection</title>
<style>
  body{{font-family:Segoe UI,Arial;background:#f8f9fa;padding:20px;margin:0}}
  h1{{color:#d93025;text-align:center}} /* Red header for PDF theme */
  input{{width:100%;padding:16px;font-size:20px;border:3px solid #d93025;border-radius:12px;box-sizing:border-box;margin:15px 0}}
  .info{{text-align:center;margin:15px;font-size:18px;color:#555}}
  .tree{{background:#fff;padding:30px;border-radius:16px;box-shadow:0 10px 30px rgba(0,0,0,0.1);font-family:Consolas,monospace;font-size:15px;line-height:1.8}}
  .fold{{cursor:pointer;color:#444;font-weight:bold}}
  .fold:hover{{color:#000}}
  .fn{{color:#d93025;text-decoration:none;font-weight:500}} /* Red links for PDFs */
  .fn:hover{{text-decoration:underline}}
  .sz{{color:#888;font-size:0.85em}}
  .dl{{margin-left:15px;color:#1a73e8;font-weight:bold;font-size:0.9em}}
  .sub{{padding-left:30px;display:block}} /* Default open rakh sakte hain ya none kar dein */
  .rootname,.shared{{font-size:20px;font-weight:bold;color:#202124;margin:20px 0 10px}}
  .hl{{background:#ffeb3b !important;padding:2px 6px;border-radius:4px}}
</style>
</head>
<body>
<h1>Google Drive - PDF Collection 📕</h1>
<input type="text" placeholder="PDF search karein..." autofocus>
<div class="info">Total PDFs & Folders: {total_items}</div>
<div class="tree">{tree_html}</div>

<script>
function t(id){{
  var x = document.getElementById(id);
  if (x.style.display === "none") {{ x.style.display = "block"; }}
  else {{ x.style.display = "none"; }}
}}
// Default sabko close rakho, taaki neat lage
document.querySelectorAll('.sub').forEach(el => el.style.display = 'none');

var input = document.querySelector('input');
var info = document.querySelector('.info');

input.oninput = function() {{
  var term = input.value.trim().toLowerCase();
  var names = document.querySelectorAll('.fn');
  var found = 0;

  names.forEach(function(el) {{
    var text = el.textContent.toLowerCase();
    if (term && text.includes(term)) {{
      el.classList.add('hl');
      found++;
      var p = el.parentElement;
      while (p) {{
        if (p.classList && p.classList.contains('sub')) p.style.display = 'block';
        p = p.parentElement;
      }}
    }} else {{
      el.classList.remove('hl');
    }}
  }});

  if (term) {{
    info.textContent = found + " PDFs mile";
    if (found > 0) document.querySelector('.hl').scrollIntoView({{behavior:'smooth', block:'center'}});
  }} else {{
    info.textContent = "Total items: {total_items}";
  }}
}};
</script>
</body>
</html>'''

with open(output_file, "w", encoding="utf-8") as f:
    f.write(html)

print("\nScanning Mukammal!")
print("Sirf PDFs aur relevant folders save kiye gaye hain.")
files.download(output_file)
display(HTML(html))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Google Drive scan kar raha hun (Sirf PDFs ke liye)...
Ab tak 35 items scan kiye...
Total items (Folders + PDFs) mile: 35
Faltu folders filter kar raha hun...
Filtering ke baad items bache: 8

Scanning Mukammal!
Sirf PDFs aur relevant folders save kiye gaye hain.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# GOOGLE DRIVE SPECIFIC FOLDER SCANNER (PDFs Only)
# Sirf aik specific folder aur uske sub-folders ko scan karega

from googleapiclient.discovery import build
from google.colab import auth, drive, files
from google.auth import default
from IPython.display import display, HTML

# --- CONFIGURATION (Yahan apni Folder ID dalain) ---
# Neeche di gayi ID ko hata kar apni Folder ID paste karein
TARGET_FOLDER_ID = "170DErJsrMpVN76SlWYvuyNNbJbWAtdQj"
# Example: "1HjK9xLmNpQrStUvWxYz"

# Step 1: Auth
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

output_file = "/content/drive/MyDrive/Folder_Specific_Tree.html"

print(f"Target Folder ID: {TARGET_FOLDER_ID}")
print("Scanning start kar raha hun... (Isme thora waqt lag sakta hai)")

def get_folder_name(folder_id):
    """Folder ka asli naam pata lagane ke liye"""
    try:
        file = service.files().get(fileId=folder_id, fields='name').execute()
        return file.get('name')
    except:
        return "Unknown Folder"

root_name = get_folder_name(TARGET_FOLDER_ID)
print(f"Main Folder Name: {root_name}")

def scan_folder_recursive(folder_id):
    """
    Ye function recursion use nahi karega, balki ek list (stack) use karega
    taaki deep folders mein error na aaye.
    """
    found_items = []
    folders_to_process = [folder_id] # Shuruwat target folder se
    processed_count = 0

    while folders_to_process:
        current_id = folders_to_process.pop(0) # List se pehla folder nikalo

        page_token = None
        while True:
            try:
                # Query: Is folder ke bachay dhundo (PDFs ya Folders)
                query = f"'{current_id}' in parents and trashed = false and (mimeType = 'application/vnd.google-apps.folder' or mimeType = 'application/pdf')"

                results = service.files().list(
                    q=query,
                    fields="nextPageToken, files(id, name, mimeType, parents, size, sharedWithMe)",
                    includeItemsFromAllDrives=True,
                    supportsAllDrives=True,
                    pageSize=1000,
                    pageToken=page_token
                ).execute()

                items = results.get('files', [])

                for item in items:
                    found_items.append(item)
                    # Agar folder hai, to isko 'folders_to_process' mein daalo taaki iske andar bhi check karein
                    if item['mimeType'] == 'application/vnd.google-apps.folder':
                        folders_to_process.append(item['id'])

                page_token = results.get('nextPageToken')
                if not page_token:
                    break
            except Exception as e:
                print(f"Error scanning folder {current_id}: {e}")
                break

        processed_count += 1
        print(f"Folders Processed: {processed_count} | Items Found: {len(found_items)}", end='\r')

    return found_items

# --- EXECUTION ---
raw_items = scan_folder_recursive(TARGET_FOLDER_ID)

# --- CLEANING (Khali folders hatana) ---
print("\nCleaning empty folders...")
item_map = {item['id']: item for item in raw_items}
relevant_ids = set()

# Sirf PDFs pakdo
pdfs = [item for item in raw_items if item['mimeType'] == 'application/pdf']

for pdf in pdfs:
    relevant_ids.add(pdf['id'])
    # Parent trace karo, lekin sirf wahan tak jahan tak humne scan kiya (TARGET_FOLDER_ID tak)
    current_parents = pdf.get('parents', [])
    while current_parents:
        pid = current_parents[0]
        # Agar parent hamare scanned items mein hai aur abhi tak add nahi hua
        if pid in item_map:
            if pid in relevant_ids: break
            relevant_ids.add(pid)
            pid_item = item_map[pid]
            current_parents = pid_item.get('parents', [])
        else:
            break

# Final List
final_items = [item for item in raw_items if item['id'] in relevant_ids]
print(f"Final Count (PDFs + Active Folders): {len(final_items)}")

# --- HTML GENERATION ---
by_id = {item['id']: item for item in final_items}
children = {}

for item in final_items:
    parents = item.get('parents', [])
    if parents:
        pid = parents[0]
        # Agar parent hamare data mein hai, to child banao
        if pid in by_id:
            children.setdefault(pid, []).append(item)
        elif pid == TARGET_FOLDER_ID:
             # Agar parent wo folder hai jo humne target kiya tha
             children.setdefault('target_root', []).append(item)

# Sort Helper
def sort_kids(kids):
    folders = [x for x in kids if x['mimeType'] == 'application/vnd.google-apps.folder']
    files_only = [x for x in kids if x['mimeType'] != 'application/vnd.google-apps.folder']
    folders.sort(key=lambda x: x['name'].lower())
    files_only.sort(key=lambda x: x['name'].lower())
    return folders + files_only

folder_counter = 0
def new_id():
    global folder_counter
    folder_counter += 1
    return f"f{folder_counter}"

def build_html(item, prefix="", is_last=True):
    name = item['name']
    fid = item['id']
    is_folder = item['mimeType'] == 'application/vnd.google-apps.folder'

    connector = "└── " if is_last else "├── "
    indent = prefix + ("    " if is_last else "│   ")

    size_text = ""
    if not is_folder and 'size' in item:
        s = int(item['size'])
        if s > 1048576: size_text = f" ({s/1048576:.1f} MB)"
        elif s > 0: size_text = f" ({s//1024} KB)"

    if is_folder:
        hid = new_id()
        line = f'{prefix}{connector}<span class="fold" onclick="t(\'{hid}\')">📂 <b>{name}</b></span><br>'
        line += f'<div id="{hid}" class="sub">'
    else:
        view = f"https://drive.google.com/file/d/{fid}/view"
        down = f"https://drive.google.com/uc?id={fid}&export=download"
        line = f'{prefix}{connector}🔴 <a href="{view}" target="_blank" class="fn">{name}</a> ' \
               f'<span class="sz">{size_text}</span> ' \
               f'<a href="{down}" download class="dl">⬇</a><br>'

    # Recursion for children
    kids = sort_kids(children.get(fid, []))
    for i, kid in enumerate(kids):
        line += build_html(kid, indent, i == len(kids)-1)

    if is_folder: line += '</div>'
    return line

# Build Tree
tree_html = f'<div class="rootname">Target Folder: {root_name}</div>'
# Start with direct children of the target folder
root_kids = sort_kids(children.get('target_root', []))

for i, kid in enumerate(root_kids):
    tree_html += build_html(kid, "", i == len(root_kids)-1)

html = f'''<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Specific Folder Scan</title>
<style>
  body{{font-family:Segoe UI, sans-serif; padding:20px; background:#f5f5f5}}
  h1{{color:#d93025; text-align:center}}
  .rootname{{font-size:22px; font-weight:bold; color:#1a73e8; margin-bottom:15px; border-bottom:2px solid #ddd; padding-bottom:10px}}
  .tree{{background:white; padding:20px; border-radius:10px; box-shadow:0 2px 10px rgba(0,0,0,0.1); font-family:Consolas, monospace; line-height:1.6}}
  .fold{{cursor:pointer; color:#333; font-weight:600}}
  .fold:hover{{color:#1a73e8}}
  .sub{{padding-left:25px; display:block}} /* Default Open */
  .fn{{color:#d93025; text-decoration:none}}
  .fn:hover{{text-decoration:underline}}
  .sz{{color:#888; font-size:0.8em}}
  .dl{{text-decoration:none; margin-left:8px}}
  input{{width:100%; padding:12px; margin-bottom:20px; border:1px solid #ccc; border-radius:5px}}
  .hl{{background:#fff9c4}}
</style>
</head>
<body>
<h1>Folder Scan: {root_name}</h1>
<input type="text" placeholder="Search in this folder...">
<div class="tree">{tree_html}</div>
<script>
function t(id){{
  var el = document.getElementById(id);
  el.style.display = (el.style.display === 'none') ? 'block' : 'none';
}}
document.querySelector('input').addEventListener('input', function(e){{
    var term = e.target.value.toLowerCase();
    document.querySelectorAll('.fn').forEach(el => {{
        var match = el.innerText.toLowerCase().includes(term);
        el.classList.toggle('hl', match && term !== "");
        if(match && term!=="") {{
            var p = el.parentElement;
            while(p.classList.contains('sub')) {{ p.style.display='block'; p=p.parentElement; }}
        }}
    }});
}});
</script>
</body>
</html>'''

with open(output_file, "w", encoding="utf-8") as f:
    f.write(html)

print(f"\nScanning Complete! File saved: {output_file}")
drive.mount('/content/drive') # Ensure mount for download
files.download(output_file)
display(HTML(html))